In [1]:
import itertools
import random

import IPython.display
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
date_csv = '20241123_190205' 
df_orig = pd.read_csv(f'../logs/rundirs/sorted-{date_csv}.csv')
df_orig

,Date,Scenario ID,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,Total distance traveled (m),No. of stops,No. of forcing events,No. of violations,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s)
0,20241123_174016,map-generator/generated-maps/2024-11-22_11:26:...,0,HumanDrivenVehicle,783.84,2,2012.7,5,0,0,32,0,0,0,147.2,70.7,599.9,0.3,5.6,3.4
1,20241123_174135,map-generator/generated-maps/2024-11-22_11:26:...,0,HumanDrivenVehicle,783.84,3,2471.5,5,6,6,40,0,1,1,71.0,39.8,599.9,0.3,5.6,4.1
2,20241123_174256,map-generator/generated-maps/2024-11-22_11:26:...,0,HumanDrivenVehicle,783.84,3,2407.6,4,5,5,39,0,0,1,107.0,104.6,599.9,0.3,5.6,4.0
3,20241123_174415,map-generator/generated-maps/2024-11-22_11:27:...,0,HumanDrivenVehicle,779.77,1,881.4,9,0,0,41,0,2,0,344.3,136.8,599.9,0.3,5.6,1.5
4,20241123_174538,map-generator/generated-maps/2024-11-22_11:27:...,0,HumanDrivenVehicle,779.77,3,2776.0,4,8,8,43,0,5,0,35.1,32.7,599.9,0.3,5.6,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,20241123_185507,map-generator/generated-maps/2024-11-22_11:26:...,4,AutonomousVehicle,604.25,2,2718.3,4,0,0,16,0,1,0,37.6,32.6,599.9,0.3,5.6,4.5
296,20241123_185631,map-generator/generated-maps/2024-11-22_11:26:...,4,AutonomousVehicle,604.25,1,2410.5,3,0,0,16,0,0,0,114.1,73.5,599.9,0.3,5.6,4.0
297,20241123_185744,map-generator/generated-maps/2024-11-22_11:27:...,4,AutonomousVehicle,604.12,2,2885.4,3,0,0,15,0,0,0,31.9,30.4,599.9,0.3,5.6,4.8
298,20241123_185911,map-generator/generated-maps/2024-11-22_11:27:...,4,AutonomousVehicle,604.12,2,2684.8,4,0,0,17,0,1,0,52.9,30.4,599.9,0.3,5.6,4.5


In [3]:
df_id = df_orig['Scenario ID'].str.split(r'[;,] ', expand=True)
df_id.columns = ['filename', 'Coordination strategy', 'string_seed', 'string_probabilityForcingForHuman', 'heuristic']
df_id = pd.concat([
    df_id,
    df_id['filename'].str.extract(r'(?P<dir_map>[^/]+)/(?P<basename_scenario>[^/]+)[.]json$', expand=True),
    df_id['filename'].str.extract(r'/scenario(?P<i_map>\d+)-(?P<i_locations>\d+)[.]json$', expand=True).astype(int),
    df_id['string_probabilityForcingForHuman'].str.extract(r'^probabilityForcingForHuman (?P<probabilityForcingForHuman>[\d.]+)$', expand=True).astype(float),
], axis=1).rename(columns={'i_locations': 'Positions variant'})
df_id['filename_screenshot'] = "../map-generator/generated-maps/" + df_id['dir_map'] + '/screenshots/' + df_id['basename_scenario'] + '.png'
df_id['are_bridges'] = df_id['dir_map'].str.contains('with_bridges')
df_id['configuration'] = df_id[
    ['i_map', 'are_bridges', 'Positions variant']].agg(
    lambda r: f'map {r['i_map']}, {"with" if r['are_bridges'] else "without"} bridges, pos.var. {r['Positions variant']}', 
    axis=1
)
df_id = df_id.sort_values(['i_map', 'are_bridges', 'Positions variant'], ascending=[True, False, True])
df_id

,filename,Coordination strategy,string_seed,string_probabilityForcingForHuman,heuristic,dir_map,basename_scenario,i_map,Positions variant,probabilityForcingForHuman,filename_screenshot,are_bridges,configuration
0,map-generator/generated-maps/2024-11-22_11:26:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,scenario1-1,1,1,0.0,../map-generator/generated-maps/2024-11-22_11:...,True,"map 1, with bridges, pos.var. 1"
1,map-generator/generated-maps/2024-11-22_11:26:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,scenario1-1,1,1,1.0,../map-generator/generated-maps/2024-11-22_11:...,True,"map 1, with bridges, pos.var. 1"
2,map-generator/generated-maps/2024-11-22_11:26:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,scenario1-1,1,1,1.0,../map-generator/generated-maps/2024-11-22_11:...,True,"map 1, with bridges, pos.var. 1"
60,map-generator/generated-maps/2024-11-22_11:26:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,scenario1-1,1,1,0.0,../map-generator/generated-maps/2024-11-22_11:...,True,"map 1, with bridges, pos.var. 1"
61,map-generator/generated-maps/2024-11-22_11:26:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:26:14_with_bridges,scenario1-1,1,1,1.0,../map-generator/generated-maps/2024-11-22_11:...,True,"map 1, with bridges, pos.var. 1"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,map-generator/generated-maps/2024-11-22_11:27:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:27:17_without_bridges,scenario1-10,1,10,1.0,../map-generator/generated-maps/2024-11-22_11:...,False,"map 1, without bridges, pos.var. 10"
239,map-generator/generated-maps/2024-11-22_11:27:...,stops,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:27:17_without_bridges,scenario1-10,1,10,1.0,../map-generator/generated-maps/2024-11-22_11:...,False,"map 1, without bridges, pos.var. 10"
297,map-generator/generated-maps/2024-11-22_11:27:...,baseline,seed 1,probabilityForcingForHuman 0,AUTOMATED_FIRST,2024-11-22_11:27:17_without_bridges,scenario1-10,1,10,0.0,../map-generator/generated-maps/2024-11-22_11:...,False,"map 1, without bridges, pos.var. 10"
298,map-generator/generated-maps/2024-11-22_11:27:...,change of priorities,seed 1,probabilityForcingForHuman 1,AUTOMATED_FIRST,2024-11-22_11:27:17_without_bridges,scenario1-10,1,10,1.0,../map-generator/generated-maps/2024-11-22_11:...,False,"map 1, without bridges, pos.var. 10"


In [4]:
configuration_to_filename_screenshot = {row['configuration']: row['filename_screenshot'] for _, row in df_id.iterrows()}
configuration_to_filename_screenshot

{'map 1, with bridges, pos.var. 1': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario1-1.png',
 'map 1, with bridges, pos.var. 2': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario1-2.png',
 'map 1, with bridges, pos.var. 3': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario1-3.png',
 'map 1, with bridges, pos.var. 4': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario1-4.png',
 'map 1, with bridges, pos.var. 5': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario1-5.png',
 'map 1, with bridges, pos.var. 6': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario1-6.png',
 'map 1, with bridges, pos.var. 7': '../map-generator/generated-maps/2024-11-22_11:26:14_with_bridges/screenshots/scenario1-7.png',
 'map 1, with bridges, pos.var. 8': '../map-generator/generated-maps/2024-11

In [5]:
index_id = df_orig.columns.get_loc('Scenario ID')
df = pd.concat([
    df_id[['i_map', 'are_bridges', 'Positions variant', 'configuration', 
           'Coordination strategy', 'probabilityForcingForHuman']], 
    df_orig.iloc[:, index_id + 1:]], 
    axis=1
)
df

,i_map,are_bridges,Positions variant,configuration,Coordination strategy,probabilityForcingForHuman,Vehicle ID,Vehicle type,Cycle distance (m),No. of completed missions,...,No. of critical sections,No. of near-misses,No. of collisions,Is blocked,Total waiting time (s),Maximum waiting time (s),Total time (s),Maximum acceleration (m/s^2),Maximum speed (m/s),Average speed (m/s)
0,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,0,HumanDrivenVehicle,783.84,2,...,32,0,0,0,147.2,70.7,599.9,0.3,5.6,3.4
1,1,True,1,"map 1, with bridges, pos.var. 1",change of priorities,1.0,0,HumanDrivenVehicle,783.84,3,...,40,0,1,1,71.0,39.8,599.9,0.3,5.6,4.1
2,1,True,1,"map 1, with bridges, pos.var. 1",stops,1.0,0,HumanDrivenVehicle,783.84,3,...,39,0,0,1,107.0,104.6,599.9,0.3,5.6,4.0
60,1,True,1,"map 1, with bridges, pos.var. 1",baseline,0.0,1,AutonomousVehicle,128.74,7,...,28,0,0,0,70.5,62.4,599.9,0.3,5.6,3.4
61,1,True,1,"map 1, with bridges, pos.var. 1",change of priorities,1.0,1,AutonomousVehicle,128.74,6,...,27,0,1,1,119.2,97.2,599.9,0.3,5.6,2.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,1,False,10,"map 1, without bridges, pos.var. 10",change of priorities,1.0,3,AutonomousVehicle,555.12,2,...,15,0,0,0,1.6,0.6,599.9,0.3,5.6,5.1
239,1,False,10,"map 1, without bridges, pos.var. 10",stops,1.0,3,AutonomousVehicle,555.12,2,...,15,0,0,0,18.2,16.6,599.9,0.3,5.6,4.9
297,1,False,10,"map 1, without bridges, pos.var. 10",baseline,0.0,4,AutonomousVehicle,604.12,2,...,15,0,0,0,31.9,30.4,599.9,0.3,5.6,4.8
298,1,False,10,"map 1, without bridges, pos.var. 10",change of priorities,1.0,4,AutonomousVehicle,604.12,2,...,17,0,1,0,52.9,30.4,599.9,0.3,5.6,4.5


In [6]:
series_blocks = df[df['Vehicle type'] != 'HumanDrivenVehicle'].groupby(['configuration', 'are_bridges'], sort=False)['Is blocked'].sum()
index_blocked = series_blocks[series_blocks != 0].index
index_nonblocked = series_blocks[series_blocks == 0].index
index_nonblocked[~index_nonblocked.get_level_values('are_bridges')].get_level_values('configuration')

Index(['map 1, without bridges, pos.var. 1',
       'map 1, without bridges, pos.var. 4',
       'map 1, without bridges, pos.var. 10'],
      dtype='object', name='configuration')

In [7]:
def save_and_show(basename):  # to avoid inlining large image data into the notebook file
    filename = f'images/{basename}.png'
    plt.savefig(filename)
    plt.close()
    # The `random` is because of https://stackoverflow.com/a/43640705.
    IPython.display.display(IPython.display.HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" />'))
    

def plot_aut_hum(df, *, title2, dfs_y1, dfs_y2, is_aut):
    #print(df)
    
    # Extract levels
    positions = df.index.get_level_values('Positions variant').unique()
    strategies = df.index.get_level_values('Coordination strategy').unique()
    
    # Plot configuration
    fig, axes = plt.subplots(1, len(positions), figsize=(20, 6), sharey=True, squeeze=False)
    axes: list[list[Axes]]
    bar_width = 0.4
    
    column_y1 = 'Total distance traveled (m)'
    color_y1 = 'tab:blue'
    
    columns_y2_aut = ['No. of collisions', 'No. of near-misses']
    colors_y2_aut = ['red', 'yellow']
    
    columns_y2_hum = ['No. of violations'] + columns_y2_aut
    colors_y2_hum = ['black'] + colors_y2_aut
    
    if is_aut:
        columns_y2 = columns_y2_aut
        colors_y2 = colors_y2_aut
        title1 = 'Automated vehicles (summarised for AV1-AV4)' 
    else:
        columns_y2 = columns_y2_hum
        colors_y2 = colors_y2_hum
        title1 = 'Human-driven vehicle'
    
    # Get the global min and max values for consistent y-axis scaling
    y1_max = max(dfx[column_y1].max() for dfx in dfs_y1)
    y2_max = max(dfx[column].max()
                 for dfx in dfs_y2
                 for column in set(columns_y2_aut + columns_y2_hum))
    
    # Iterate through each Positions variant
    for i, position in enumerate(positions):
        ax = axes[0][i]
        df_pos = df.loc[position]
        
        # Bar positions for each Coordination strategy
        x_positions = np.arange(len(strategies))
        
        # Plot bars
        handles = [ax.bar(x_positions, df_pos[column_y1], width=bar_width, label=column_y1, color=color_y1)[0]]
        ax.set_xlabel('Coordination Strategy')
        ax.set_ylabel(column_y1, color=color_y1)
        ax.tick_params(axis='y', labelcolor=color_y1)
        ax.set_ylim(0, y1_max * 1.1)
        
        # Create a secondary axis for the points
        ax_right = ax.twinx()
        
        # Plot points
        handles += [ax_right.plot(x_positions, df_pos[column], label=column,
                                  marker='o', linestyle='', color=color)[0]
                    for column, color in zip(columns_y2, colors_y2)]
        ax_right.tick_params(axis='y', labelcolor='black')
        ax_right.set_ylim(0, y2_max * 1.1)
        
        # Add labels, grid, and title for each section
        ax.set_xticks(x_positions)
        ax.set_xticklabels(strategies, rotation=45, ha='right')
        ax.set_title(f'Position {position}')
        ax.grid(axis='y')
        
    fig.suptitle(f'{title1}\n{title2}', fontsize=16)
    fig.legend(handles=handles, labels=[column_y1, *columns_y2], ncol=len(handles), loc='upper right')
    plt.tight_layout()
    save_and_show(f'{title1}: {title2}')
    #print(id(df))
    #IPython.display.display(df)


key2df = {}
for are_bridges in True, False:
    for is_aut in True, False:
        dfx = df[df['are_bridges'] == are_bridges]
        dfx = dfx[dfx['configuration'].isin(index_nonblocked.get_level_values('configuration'))]
        dfx = dfx[dfx['Vehicle type'] == ('AutonomousVehicle' if is_aut else 'HumanDrivenVehicle')]
        dfx = dfx.groupby(['Positions variant', 'Coordination strategy']).sum()
        key2df[are_bridges, is_aut] = dfx    
        
for are_bridges in True, False:
    for is_aut in True, False:
        try:
            plot_aut_hum(key2df[are_bridges, is_aut],
                         title2='With bridges' if are_bridges else 'Without bridges', 
                         dfs_y1=[dfx for (_, is_aut_dfx), dfx in key2df.items() if is_aut_dfx == is_aut], 
                         dfs_y2=key2df.values(), 
                         is_aut=is_aut)
        except Exception as exc:
            print(exc)
            #raise exc
    print()

In [8]:
def show_maps(title, configurations, ncols): 
    nrows = max(1, (len(configurations) + ncols - 1) // ncols)
    fig, axes_matrix = plt.subplots(nrows, ncols, figsize=(16, 9), squeeze=False)
    print(f'{title}: {nrows}x{ncols}')

    axes = list(itertools.chain.from_iterable(axes_matrix))
    axes_matrix: list[list[Axes]]
    assert len(axes) >= len(configurations)
    
    for ax in axes:
        ax.axis('off')
        
    for ax, configuration in zip(axes, configurations):
        filename_screenshot = configuration_to_filename_screenshot[configuration]
        try:
            image = plt.imread(filename_screenshot)
            ax.imshow(image)
        except Exception as exc:
            print(exc)
        ax.title.set_text(f'Configuration:\n{configuration}')
    
    fig.suptitle(title, fontsize=16)    
    fig.tight_layout()
    fig.subplots_adjust(wspace=0.1, hspace=0.3)
    save_and_show(title)
    
    
for title, index in {'Non-blocked': index_nonblocked, 'Blocked': index_blocked}.items():
    for are_bridges in True, False:
        show_maps(f'{title} ({"with" if are_bridges else "without"} bridges)', 
                  index[index.get_level_values('are_bridges') == are_bridges].get_level_values('configuration'), 
                  4)

Non-blocked (with bridges): 2x4


Non-blocked (without bridges): 1x4


Blocked (with bridges): 1x4


Blocked (without bridges): 2x4
